In [2]:
import polars as pl
import os
import importlib

from pattern_detector import utils


### Calculate patterns

In [6]:
filepath = os.path.join("data", "silver", "combined.parquet")
dates = pl.read_parquet(filepath, columns="date").to_series()

all_patterns = utils.generate_possible_patterns(dates)

NameError: name 'utils' is not defined